In [1]:
import numpy as np
import pandas as pd
import spacy
import re
import string
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [7]:
cv = CountVectorizer()

df = pd.read_csv('Meta.csv', sep=';', keep_default_na=False, low_memory=False, on_bad_lines='error')

#creo una colonna che contiene tutte le parti di testo analizzabili

#df['complessiva']=df['Message'].astype(str)+' ' +df['Image Text'].astype(str)+' '+df['Link Text'].astype(str)+ ' '+df['Description'].astype(str)
df['Tot'] = df['word'].astype(str)

nlp = spacy.load('it_core_news_lg')
nlp_en = spacy.load('en_core_web_lg')
nlp_es = spacy.load('es_core_news_lg')
nlp_fr = spacy.load('fr_core_news_lg')
nlp_de = spacy.load('de_core_news_lg')
nlp_pt = spacy.load('pt_core_news_lg')


#metodo per rimuovere siboli e emoticons
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u000D" #ritorno a capo
                               u"\u000A" #newline
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               u"\u0023" #cancelletto
                               u"\u0030-\u0039"#numeri
                               u"\u00A0-\u00BB" #rimozione virgolette strane
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', string)

#metodo non usato al momento
def tokenize(texts):
    """
    Tokenizes a list of texts, returning a list of tokens.
    """
    token_texts = []
    for text in texts:
        token_texts.append(simple_preprocess(text, deacc=True))
    return token_texts


#metodo per creare i lemmi, contiene una lista di stopword ulteriore
def lemming_cell(text):
    stopword=['nn','settembrese','x','xke','codvid','di','a','da','in','con','contro','coi','su',
              'sui','sullo','sulle','sulla','sugli','sull','per','tra','fra','ho','sono','ad','allo','agli',
              'alla','alle','allo','all','col','dal','dei','dallo','dalla','dalle','dagli','dell','il','lo',
              'la','i','gli','le','un','uno','una','io','tu','lui','lei','noi','voi','loro','mio','mia',
              'miei','mie','tuo','tua','tuoi','tue','suo','sua','suoi','già','gia','eh','cè','ce','poi','boh',
              'sue','nostro','nostra','nostri','nostre','vostro','vostra','vostri','vostre','mi','ti','ci',
              'vi','ma','ed','se','perché','anche','come','dov','dove','che','chi','cui','non','più','quale',
              'quanto','quanti','quanta','quante','quello','quelli','quella','quelle','questo','questi',
              'questa','queste','si','no','non','tutto','tutti','a','c','e','i','l','o','ho','hai','ha',
              'abbiamo','avete','hanno','abbia','abbiate','abbiano','avere','avrò','avro','avrai','avrà',
              'avremo','avrete','avranno','avrei','avresti','avrebbe','avremmo','avreste','avrebbero','avevo',
              'avevi','aveva','avevamo','avevate','avevano','ebbi','avesti','ebbe','avemmo','aveste','ebbero',
              'avessi','avesse','avessimo','avessero','avendo','avuto','avuta','avuti','avute','essere','sono',
              'sei','è','siamo','siete','sia','siate','siano','sarò','saro','sarai','sarà','sara','saremo',
              'sarete','saranno','sarei','saresti','sarebbe','saremmo','sareste','sarebbero','ero','eri','era',
              'eravamo','eravate','erano','fui','fosti','fu','fummo','foste','furono','fossi','fosse','fossimo',
              'fossero','essendo','faccio','fai','facciamo','fanno','faccia','facciate','facciano','farò',
              'farai','farà','fara','faremo','fare','farete','faranno','farei','faresti','farebbe','faremmo',
              'fareste','farebbero','facevo','facevi','faceva','facevamo','facevate','facevano','feci',
              'facesti','fece','facemmo','faceste','fecero','facessi','facesse','facessimo','facessero',
              'facendo','sto','stai','sta','stiamo','stanno','stia','stiate','stiano','starò','starai','starà',
              'staremo','starete','staranno','starei','staresti','starebbe','staremmo','stareste','starebbero',
              'stavo','stavi','stava','stavamo','stavate','stavano','stetti','stesti','stette','stemmo',
              'steste','stettero','stessi','stesse','stessimo','stessero','stando','dare','diedi','dai',
              'daremo','darete','davo','davi','dava','davamo','davate','davano','dasti','dassero','dando',
              'davasti','desti','davammo','davaste','diedero','leggere','letto','leggo','leggiamo','photos',
              'leggete','leggono','leggevo','leggeva','leggevi','leggevamo','leggevate','leggevano','leggendo',
              'mantenere','manteniamo','mantengo','mantieni','mantiene','mantenete','mantengono','mantenevo',
              'mantenevi','manteneva','mantenuto','mantenendo','rispettare','rispetto','rispetti','rispetta',
              'rispettiamo','rispettate','rispettano','rispettavo','rispettavi','rispettava','rispettato',
              'intervistare','intervistato','intervistavano','andare','andavo','andato','andai','vado','vai',
              'va','andiamo','andate','vanno','andando','andava','andavi','arrivare','numerare','mettere',
              'messo','metto','sapere','fino','così','cosi','bho','to','good','sorry','shitcoinmoonshots','gt','removed','deleted','concerns','mora',
              'going','it','want','lot','actually','moro','way','right','going','amp','sure','ll','lot','re',
              'gonna','wow','lol','delete','oh','damn','it','ve','didn','said','ar','fuck','doesn','you','i',
              'are','am','dude','rufus','gt','merda','voc','ms','cmo','sem','tem','uma','notcias', 'usare','iebs',
              'cosa','estarn','inversin','cardare','lanamento','minha','kazu','davvero']
   
    stopword_en = nlp_en.Defaults.stop_words
    stopword_es = nlp_es.Defaults.stop_words
    stopword_fr = nlp_fr.Defaults.stop_words
    stopword_de = nlp_de.Defaults.stop_words
    stopword_pt = nlp_pt.Defaults.stop_words


    
    
    tokens=[] 
    doc=nlp(text.lower()) #trasforma tutte le parole in minuscolo in minuscolo
    
    for token in doc:
        # se la parola non è un URL e non è nella lista stop word, allora mette il lemma nel vettore "tokens"
        if token.pos_ in ['PROPN','NOUN','VERB','ADJ','ADV'] and not token.is_stop and not token.like_url and token.lemma_ not in stopword and token.lemma_ not in stopword_en and token.lemma_ not in stopword_es and token.lemma_ not in stopword_fr and token.lemma_ not in stopword_de and token.lemma_ not in stopword_pt:
            tokens.append(token.lemma_)
    tokens=" ".join(set(tokens)) # trasforma vettore tokens in una stringa
    return tokens
"""
#costruisco un file con le colonne riunite ma ripulite per TLAB
finale=open("dati.csv","w")
for i in df['complessiva']:
    if len(i)<2000:
        pulito=remove_emoji(str(i))
        a=pulito.replace('  ',' ')
        a=a.replace('\n','')
        finale.write(a)
        finale.write('\n')
        
finale.close() 
"""
#costruisco un corpus con i singoli post già puliti da emoticon, da segni di interpunzione e url

corpus=[] # corpus è un vettore di stringhe "pulite"
for i in df['Tot']:
    phrase=remove_emoji(str(i))
    vett=lemming_cell(phrase)
    if vett !="":
        corpus.append(vett)

In [8]:
dtm = cv.fit_transform(corpus)
LDA = LatentDirichletAllocation(n_components=5,n_jobs=-1) # funzione per specificare il numero di cluster
LDA.fit(dtm)


#len(cv.get_feature_names())
for index,topic in enumerate(LDA.components_):
    print(f'TOP 30 PAROLE DEL CLUSTER #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-30:]])
    print('\n')

TOP 30 PAROLE DEL CLUSTER #0
['year', 'times', 'reason', 'avatar', 'kind', 'question', 'start', 'speech', 'pretty', 'wrong', 'yeah', 'play', 'humanity', 'movie', 'hope', 'online', 'big', 'idea', 'fb', 'tech', 'interview', 'player', 'ready', 'great', 'futuro', 'reality', 'social', 'video', 'mark', 'metaverse']


TOP 30 PAROLE DEL CLUSTER #1
['crazy', 'fucking', 'fun', 'content', 'vice', 'questions', 'instagram', 'amazing', 'remember', 'china', 'interesting', 'ill', 'basically', 'phone', 'hard', 'wait', 'fact', 'watching', 'sounds', 'watch', 'evil', 'making', 'wont', 'thought', 'talk', 'matrix', 'talking', 'work', 'live', 'meta']


TOP 30 PAROLE DEL CLUSTER #2
['generation', 'word', 'started', 'level', 'oculus', 'bro', 'turn', 'business', 'conversation', 'lizard', 'account', 'trust', 'news', 'coming', 'true', 'guys', 'youtube', 'fake', 'problem', 'platform', 'ben', 'literally', 'control', 'hate', 'company', 'bad', 'technology', 'zuckerberg', 'life', 'real']


TOP 30 PAROLE DEL CLUSTER #3

In [9]:
# Creazione grafico

from pyLDAvis import sklearn as sklearn_lda
import pickle 
import pyLDAvis
import os
number_topics=5
count_data = dtm

A=len(cv.get_feature_names_out())

LDAvis_data_filepath = os.path.join('./Meta'+str(number_topics))

# this is a bit time consuming - make the if statement True
# if you want to execute visualization prep yourself

if 1 == 1:
    LDAvis_prepared = sklearn_lda.prepare(LDA, dtm, cv, mds='mmds')
    #print (LDAvis_prepared)
with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared,f,0)
        
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './Meta'+ str(number_topics) +'.html')

count_data = cv.fit_transform(df['Tot'])
print(count_data)

/Users/Emanuele/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/Emanuele/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/Emanuele/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/Emanuele/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alt

/Users/Emanuele/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/Emanuele/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/Emanuele/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/Emanuele/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/Emanuele/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version clas

  (0, 35857)	1
  (1, 35857)	1
  (2, 35856)	1
  (3, 50858)	1
  (4, 70434)	1
  (5, 79925)	1
  (6, 65718)	1
  (7, 18773)	1
  (8, 44719)	1
  (9, 16086)	1
  (10, 63340)	1
  (11, 26999)	1
  (12, 12686)	1
  (13, 62280)	1
  (14, 13124)	1
  (15, 8332)	1
  (16, 75718)	1
  (17, 45617)	1
  (18, 44131)	1
  (19, 50858)	1
  (20, 70434)	1
  (21, 79925)	1
  (22, 65718)	1
  (23, 18773)	1
  (24, 44719)	1
  :	:
  (2014954, 48229)	1
  (2014955, 58218)	1
  (2014956, 65467)	1
  (2014957, 79538)	1
  (2014958, 41697)	1
  (2014959, 81442)	1
  (2014960, 71993)	1
  (2014961, 4932)	1
  (2014962, 4174)	1
  (2014963, 21089)	1
  (2014964, 12174)	1
  (2014965, 58850)	1
  (2014966, 1350)	1
  (2014967, 48229)	1
  (2014968, 58218)	1
  (2014969, 65467)	1
  (2014970, 79538)	1
  (2014971, 4932)	1
  (2014972, 19236)	1
  (2014973, 45387)	1
  (2014974, 52231)	1
  (2014975, 4932)	1
  (2014976, 19236)	1
  (2014977, 45387)	1
  (2014978, 52231)	1


In [ ]:
dataset = pd.DataFrame(corpus)
dataset.to_excel("Dataset_Reddit_EN.xlsx")